### lncRNAs quantified? relative to background
1. construct backgorund gtf
2. count reads with rnaseqc
3. compare to see if lncRNAs are quantified

In [1]:
import pandas as pd
import seaborn as sns
import pyranges as pr
import numpy as np
import bisect

from tqdm import tqdm
tqdm.pandas()

ModuleNotFoundError: No module named 'pandas'

#### create background gtf

bed files for exons and background are created with `scripts/analysis/create_matched_nongenic_null.sh`

In [ ]:
exons = pd.read_csv("/oak/stanford/groups/smontgom/klawren/nph_gtex/data/tmp/macaques/exons_to_match.bed", sep='\t', header=None, names=['chr', 'start', 'end', 'exon_id', 'gene_id'])
exons['length'] = exons['end'] - exons['start']
exons = exons[~exons['chr'].isin(['chrM'])]

# if introns are allowed
#background = pd.read_csv("/home/klawren/oak/gtex/data/other_references/nongenic_null/background_regions.bed", sep='\t', header=None, names=['chr', 'start', 'end'])
# if introns aren't allowed 
background = pd.read_csv("/oak/stanford/groups/smontgom/klawren/nph_gtex/data/tmp/macaques/intergenic_regions.bed", sep='\t', header=None, names=['chr', 'start', 'end'])

background = background[background['chr'].isin(exons['chr'].unique())]
background = background.sort_values(['chr', 'start'])

In [ ]:
# load in gencode 47 gtf
gencode_v47_path = "/oak/stanford/groups/smontgom/dnachun/data/dgtex/dGTEx_Consortium_resources/NHP-dGTEx_references/Enhanced_GTFs/Macaques/v1/Macaque_Mmul10_enhanced_v1_genes_collapsed_stranded.gtf"
gencode_v47=pr.read_gtf(gencode_v47_path)
gencode_v47_df = gencode_v47.as_df()
gencode_v47_df = gencode_v47_df[~(gencode_v47_df['Chromosome'] == 'chrM')]

In [ ]:
# def get_closest_background(exon, background_starts, background_ends):
#     exon_length = exon['end'] - exon['start']
#     idx_right = bisect.bisect_left(background_starts, exon['start'])
#     # catch index too far left
#     while idx_right < len(background_starts) and background_starts[idx_right] < exon['start']:
#         idx_right += 1

#     idx_left = idx_right - 1
#     max_iter = len(background_starts)  # Safety limit
#     iter_count = 0

#     while iter_count < max_iter:
#         iter_count += 1
#         # Try right side
#         if idx_right < len(background_starts):
#             right_start, right_end = background_starts[idx_right], background_ends[idx_right]
#             if right_end - right_start >= exon_length:
#                 return([right_start, right_start + exon_length])
        
#         # Try left side
#         if idx_left >= 0:
#             left_start, left_end = background_starts[idx_left], background_ends[idx_left]
#             if left_end - left_start >= exon_length:
#                 return([left_end-exon_length, left_end])  
        
#         idx_right += 1
#         idx_left -= 1

In [ ]:
# null_exons = []
# for chrom, background_chrom in background.groupby('chr'):
#     null_exons_chr = []
#     print(chrom)
#     background_starts = background_chrom['start'].values
#     background_ends = background_chrom['end'].values
#     exon_chrom = exons[exons['chr'] == chrom]
#     print(len(exon_chrom))
#     for idx, exon in tqdm(exon_chrom.iterrows(), total=len(exon_chrom)):
#         null_exons_chr.append(get_closest_background(exon, background_starts, background_ends))
#     null_exons_chr = pd.DataFrame(null_exons_chr, columns=['start', 'end'])
#     null_exons_chr['exon_id'] = exon_chrom['exon_id'].values
#     null_exons_chr['gene_id'] = exon_chrom['gene_id'].values
#     null_exons_chr['chr'] = chrom
#     null_exons.append(null_exons_chr)

# # concat and save
# null_exons_all_chrs = pd.concat(null_exons)
# null_exons_all_chrs[['chr', 'start', 'end', 'exon_id', 'gene_id']].to_csv('/home/klawren/oak/gtex/data/other_references/nongenic_null/matched_nongenic_null_intergenic.bed', index=False, header=None, sep = '\t')

"each gene have a single transcript whose id matches the parent gene id" from rnaseqc

In [ ]:
# # fix the null name to be compatible
# null_exons_all_chrs['exon_id'] = null_exons_all_chrs['exon_id'].str.split(';').str[0]

# # merge in the null information
# null_exons_gtf = pd.merge(gencode_v47_df[gencode_v47_df['Feature'] == 'exon'].reset_index(), null_exons_all_chrs, on='exon_id', suffixes=['', '_null'])
# # combine into the main gtf based on index 
# null_full_gtf = pd.merge(gencode_v47_df.reset_index(), null_exons_gtf[['index', 'start', 'end']], on='index', how='left', suffixes=['', '_null'])

# # add gene starts and ends as bounds 
# gene_starts = null_full_gtf.groupby('gene_id').agg({'start':'min', 'end':'max'})
# null_full_gtf = pd.merge(null_full_gtf, gene_starts, on='gene_id', how='left', suffixes=['', '_gene'])

# # replace the start and end with the null version
# null_full_gtf['Start'] = np.where(null_full_gtf['Feature'] == 'exon', null_full_gtf['start'], null_full_gtf['start_gene'])
# null_full_gtf['End'] = np.where(null_full_gtf['Feature'] == 'exon', null_full_gtf['end'], null_full_gtf['end_gene'])

In [ ]:
# # Function to convert to gtf
# def df_to_gtf(df, output_file):
#     with open(output_file, 'w') as f:
#         f.write("##description: nongenic nulls matched to exons from annotation of the human genome (GRCh38), version 47 (Ensembl 113)\n##provider: Kate Lawrence\n##contact: klawren@stanford.edu\n##format: gtf\n##date: 2025-03-21")
#         for index, row in df.iterrows():
#             attributes = f'gene_id "{row["gene_id"]}"; transcript_id "{row["transcript_id"]}"; exon_id "{row["exon_id"]}";'
#             gtf_line = f'{row["Chromosome"]}\t{row["Source"]}\t{row["Feature"]}\t{row["Start"]}\t{row["End"]}\t{row["Score"]}\t{row["Strand"]}\t{row["Frame"]}\t{attributes}\n'
#             f.write(gtf_line)

# df_to_gtf(null_full_gtf, '/home/klawren/oak/gtex/data/other_references/nongenic_null/matched_nongenic_null_intergenic.v47.gtf')

### dont allow exon overlap

In [ ]:
def get_closest_background_no_overlap(exon, background_starts, background_ends):
    exon_length = exon['end'] - exon['start']
    idx_right = bisect.bisect_left(background_starts, exon['start'])
    # catch index too far left
    while idx_right < len(background_starts) and background_starts[idx_right] < exon['start']:
        idx_right += 1

    idx_left = idx_right - 1
    max_iter = len(background_starts)  # Safety limit
    iter_count = 0

    while iter_count < max_iter:
        iter_count += 1
        # Try right side
        if idx_right < len(background_starts):
            right_start, right_end = background_starts[idx_right], background_ends[idx_right]
            if right_end - right_start >= exon_length:
                background_starts[idx_right] = background_starts[idx_right] + exon_length
                return([right_start, right_start + exon_length])
        
        # Try left side
        if idx_left >= 0:
            left_start, left_end = background_starts[idx_left], background_ends[idx_left]
            if left_end - left_start >= exon_length:
                background_ends[idx_left] = background_ends[idx_left] - exon_length
                return([left_end-exon_length, left_end])  
        
        idx_right += 1
        idx_left -= 1

In [ ]:
null_exons = []
for chrom, background_chrom in background.groupby('chr'):
    null_exons_chr = []
    print(chrom)
    background_starts = background_chrom['start'].values
    background_ends = background_chrom['end'].values
    exon_chrom = exons[exons['chr'] == chrom]
    print(len(exon_chrom))
    for idx, exon in tqdm(exon_chrom.iterrows(), total=len(exon_chrom)):
        null_exons_chr.append(get_closest_background_no_overlap(exon, background_starts, background_ends))
    null_exons_chr = pd.DataFrame(null_exons_chr, columns=['start', 'end'])
    null_exons_chr['exon_id'] = exon_chrom['exon_id'].values
    null_exons_chr['gene_id'] = exon_chrom['gene_id'].values
    null_exons_chr['chr'] = chrom
    null_exons.append(null_exons_chr)

# concat and save
null_exons_all_chrs = pd.concat(null_exons)
null_exons_all_chrs[['chr', 'start', 'end', 'exon_id', 'gene_id']].to_csv('/oak/stanford/groups/smontgom/klawren/nph_gtex/data/tmp/macaques/matched_nongenic_null_intergenic_no_overlap.bed', index=False, header=None, sep = '\t')

In [ ]:
# fix the null name to be compatible
null_exons_all_chrs['exon_id'] = null_exons_all_chrs['exon_id'].str.split(';').str[0]

# merge in the null information
null_exons_gtf = pd.merge(gencode_v47_df[gencode_v47_df['Feature'] == 'exon'].reset_index(), null_exons_all_chrs, on='exon_id', suffixes=['', '_null'])
# combine into the main gtf based on index 
null_full_gtf = pd.merge(gencode_v47_df.reset_index(), null_exons_gtf[['index', 'start', 'end']], on='index', how='left', suffixes=['', '_null'])

# add gene starts and ends as bounds 
gene_starts = null_full_gtf.groupby('gene_id').agg({'start':'min', 'end':'max'})
null_full_gtf = pd.merge(null_full_gtf, gene_starts, on='gene_id', how='left', suffixes=['', '_gene'])

# replace the start and end with the null version
null_full_gtf['Start'] = np.where(null_full_gtf['Feature'] == 'exon', null_full_gtf['start'], null_full_gtf['start_gene'])
null_full_gtf['End'] = np.where(null_full_gtf['Feature'] == 'exon', null_full_gtf['end'], null_full_gtf['end_gene'])

In [ ]:
# Function to convert to gtf
def df_to_gtf(df, output_file):
    with open(output_file, 'w') as f:
        f.write("##description: nongenic nulls matched to exons from annotation of the human genome (GRCh38), version 47 (Ensembl 113)\n##provider: Kate Lawrence\n##contact: klawren@stanford.edu\n##format: gtf\n##date: 2025-03-21")
        for index, row in df.iterrows():
            attributes = f'gene_id "{row["gene_id"]}"; transcript_id "{row["transcript_id"]}"; exon_id "{row["exon_id"]}";'
            gtf_line = f'{row["Chromosome"]}\t{row["Source"]}\t{row["Feature"]}\t{row["Start"]}\t{row["End"]}\t{row["Score"]}\t{row["Strand"]}\t{row["Frame"]}\t{attributes}\n'
            f.write(gtf_line)

df_to_gtf(null_full_gtf, '/oak/stanford/groups/smontgom/klawren/nph_gtex/references/macaques_null_intergenic.gtf')

### matched null for v10

In [ ]:
# exons = pd.read_csv("/home/klawren/oak/gtex/data/other_references/nongenic_null_v10/exons_to_match.bed", sep='\t', header=None, names=['chr', 'start', 'end', 'exon_id', 'gene_id'])
# exons['length'] = exons['end'] - exons['start']
# exons = exons[~exons['chr'].isin(['chrM'])]

# # if introns are allowed
# #background = pd.read_csv("/home/klawren/oak/gtex/data/other_references/nongenic_null/background_regions.bed", sep='\t', header=None, names=['chr', 'start', 'end'])
# # if introns aren't allowed 
# background = pd.read_csv("/home/klawren/oak/gtex/data/other_references/nongenic_null_v10/intergenic_regions.bed", sep='\t', header=None, names=['chr', 'start', 'end'])

# background = background[background['chr'].isin(exons['chr'].unique())]
# background = background.sort_values(['chr', 'start'])

# # load in gencode 39 gtf
# gencode_v39_path = "/home/klawren/oak/gtex/data/realign_references/gencode.v39.genes.gtf"
# gencode_v39=pr.read_gtf(gencode_v39_path)
# gencode_v39_df = gencode_v39.as_df()
# gencode_v39_df = gencode_v39_df[~(gencode_v39_df['Chromosome'] == 'chrM')]

In [ ]:
# null_exons = []
# for chrom, background_chrom in background.groupby('chr'):
#     null_exons_chr = []
#     print(chrom)
#     background_starts = background_chrom['start'].values
#     background_ends = background_chrom['end'].values
#     exon_chrom = exons[exons['chr'] == chrom]
#     print(len(exon_chrom))
#     for idx, exon in tqdm(exon_chrom.iterrows(), total=len(exon_chrom)):
#         null_exons_chr.append(get_closest_background_no_overlap(exon, background_starts, background_ends))
#     null_exons_chr = pd.DataFrame(null_exons_chr, columns=['start', 'end'])
#     null_exons_chr['exon_id'] = exon_chrom['exon_id'].values
#     null_exons_chr['gene_id'] = exon_chrom['gene_id'].values
#     null_exons_chr['chr'] = chrom
#     null_exons.append(null_exons_chr)

# # concat and save
# null_exons_all_chrs = pd.concat(null_exons)
# null_exons_all_chrs[['chr', 'start', 'end', 'exon_id', 'gene_id']].to_csv('/home/klawren/oak/gtex/data/other_references/nongenic_null_v10/matched_nongenic_null_intergenic_no_overlap.bed', index=False, header=None, sep = '\t')

In [ ]:
# # fix the null name to be compatible
# null_exons_all_chrs['exon_id'] = null_exons_all_chrs['exon_id'].str.split(';').str[0]

# # merge in the null information
# null_exons_gtf = pd.merge(gencode_v39_df[gencode_v39_df['Feature'] == 'exon'].reset_index(), null_exons_all_chrs, on='exon_id', suffixes=['', '_null'])
# # combine into the main gtf based on index 
# null_full_gtf = pd.merge(gencode_v39_df.reset_index(), null_exons_gtf[['index', 'start', 'end']], on='index', how='left', suffixes=['', '_null'])

# # add gene starts and ends as bounds 
# gene_starts = null_full_gtf.groupby('gene_id').agg({'start':'min', 'end':'max'})
# null_full_gtf = pd.merge(null_full_gtf, gene_starts, on='gene_id', how='left', suffixes=['', '_gene'])

# # replace the start and end with the null version
# null_full_gtf['Start'] = np.where(null_full_gtf['Feature'] == 'exon', null_full_gtf['start'], null_full_gtf['start_gene'])
# null_full_gtf['End'] = np.where(null_full_gtf['Feature'] == 'exon', null_full_gtf['end'], null_full_gtf['end_gene'])

In [ ]:
# # Function to convert to gtf
# def df_to_gtf(df, output_file):
#     with open(output_file, 'w') as f:
#         f.write("##description: nongenic nulls matched to exons from annotation of the human genome (GRCh38), version 39\n##provider: Kate Lawrence\n##contact: klawren@stanford.edu\n##format: gtf\n##date: 2025-03-28")
#         for index, row in df.iterrows():
#             attributes = f'gene_id "{row["gene_id"]}"; transcript_id "{row["transcript_id"]}"; exon_id "{row["exon_id"]}";'
#             gtf_line = f'{row["Chromosome"]}\t{row["Source"]}\t{row["Feature"]}\t{row["Start"]}\t{row["End"]}\t{row["Score"]}\t{row["Strand"]}\t{row["Frame"]}\t{attributes}\n'
#             f.write(gtf_line)

# df_to_gtf(null_full_gtf, '/home/klawren/oak/gtex/data/realign_references/gencode.v39.matched_nongenic_null_intergenic.gtf')